# Практическое задание

**Настройка общих параметров модели через конфигурационный файл default.yaml**

**Установите в общем конфигурационном файле default.yaml параметры, рекомендуемые преподавателями для запуска обучения модели: epochs 50, batch 16, imgsz 640.**

**Загрузите отредактированный файл default.yaml на Платформу.**

## Сруктура проекта 

**Для проекта используется следующая структура**

### Файл настроек используется для задания конфигураций обучений, проверки и инференса модели

#### Содержимое файле **default.yaml** - файл настроек:
**Основные параметры**:
* Задача обучения - детекция
* Модель - предобученная, yolo8n
* Обучение на 20 эпохах
* Время эпох переопределения эпох
* Размер батча - 16
* Размер изображения 640х640
* Заданы гиперпараметры:
  * lr0=lrf=0,01 (lerning rate не меняется в процессе обучения);
  * momentum = 0,937;
  * оптимизатор весов = 0.0005 и др.

### Запуск обучения с использованием конфигурационного файла

In [4]:
model_npt = YOLO("yolov8n.pt")
model_npt.train(data='data.yaml', cfg='default.yaml')